In [ ]:
import pandas as pd
import numpy as np
import os, sys
from sklearn.model_selection import train_test_split as tts
from statsmodels.tsa.arima.model import ARIMA as a
from statsmodels.stats.diagnostic import acorr_ljungbox as l
import warnings

warnings.filterwarnings("ignore")

df = pd.read_csv(os.path.join(sys.path[0], input()))

df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace = True)
df.dropna(inplace = True)

tr, te = tts(pd.to_numeric(df['Power_Consumption_diff']).dropna(), test_size = 0.2, shuffle = False)

print(f"Training data size: {len(tr)}")
print(f"Testing data size: {len(te)}")
print()

bm = np.inf
o = None
order = None

for i in range(1,6):
    model = a(tr, order = (i,0,0)).fit()
    print(f"AR({i}) AIC: {model.aic}")
    if(bm > model.aic):
        bm = model.aic
        order = (i,0,0)
        o = f"AR({i})"
        
for i in range(1,6):
    model = a(tr, order = (0,0,i)).fit()
    print(f"MA({i}) AIC: {model.aic}")
    if(bm > model.aic):
        bm = model.aic
        order = (0,0,i)
        o = f"MA({i})"
        
print()
print("Best Model Selected:")
print(o)
print()

m = a(tr, order = order).fit()
print(m.summary())
print()

print("Ljung-Box Test Results:")
print(l(m.resid,lags=1, return_df=True))